# napari fireworks

In [1]:
import napari

import numpy as np

$x=v_0 t \cos (\theta) \\
y=v_0 t \sin (\theta) - \frac{1}{2}gt^2$

In [2]:
rng = np.random.default_rng()

In [3]:
def firework(
    x0: float, 
    y0: float, 
    t0: float, 
    v0: float, 
    theta: float, 
    *, 
    n: int = 100
) -> np.ndarray:
    """Parabolic trajectory."""
    g = 9.8
    
    t = np.arange(t0, t0+n)
    dt = t - t0
    x = x0 + v0*dt*np.cos(theta)
    y = y0 + v0*dt*np.sin(theta) - (0.5*g*dt**2)
    
    y = np.clip(y, 0, np.inf)
    
    return np.stack([t, y, x], axis=-1)

### simulate a fireworks display

In [4]:
N_FIREWORKS = 500
N_SPARKS_PER_FIREWORK = 30

data = []
idx = 1

for i in range(N_FIREWORKS):
    
    # simulate a firework and append it to the track list
    xy = (0, 0)
    t = rng.integers(0, 300)
    sgn = rng.choice([-1, 1])
    theta = rng.uniform(np.pi/3, np.pi/2) 
    v = rng.uniform(1000., 3000.)
    
    d = firework(*xy, t, v, theta)
    d[:, -1] = d[:, -1] * sgn
    d = np.concatenate([np.ones((d.shape[0], 1)) * idx, d], axis=1)
    data.append(d)
    
    # for each firework, pick a point and initialize a bunch of fireworklets (sparks)
    ridx = rng.integers(90, 100)
    p = d[ridx, :]
    
    for j in range(N_SPARKS_PER_FIREWORK):
        idx += 1
        theta = rng.uniform(np.pi/4, np.pi/2)
        sgn = rng.choice([-1, 1])
        v = rng.uniform(100., 500.)
        _d = firework(p[3], p[2], p[1], v, theta, n=100)
        _d[:, -1] = _d[:, -1] * sgn
        _d = np.concatenate([np.ones((_d.shape[0], 1)) * idx, _d], axis=1)
        data.append(_d)
    
    idx+=1
    
data = np.concatenate(data, axis=0)

data[:, 2] = -data[:, 2]

### visualize it in napari using the Tracks layer

In [6]:
viewer = napari.Viewer()
viewer.add_tracks(data, name="fireworks")

<Tracks layer 'fireworks' at 0x16d772ee0>